Here we can declare a parameterized function that queries the model for a specific well and time range

In [ ]:
spark.sql("""
    CREATE OR REPLACE FUNCTION shm.3w.get_hydrate_predictions(
        p_well_number INT DEFAULT 16 COMMENT "Well number between 1 and 50, must be integer", 
        p_num_minutes INT DEFAULT 1 COMMENT "Number of minutes to predict, must be under 30"
    )
    RETURNS TABLE (
        timestamp TIMESTAMP,
        prediction DOUBLE 
    )
    COMMENT "Generates hydrate predictions for the next couple of minutes"
    RETURN
    SELECT 
        f.timestamp,
        AI_QUERY(
            'hydrate_xgboost',
            request => NAMED_STRUCT(
            'T-JUS-CKP', f.`T-JUS-CKP`,
            'T-TPT', f.`T-TPT`,
            'P-TPT', f.`P-TPT`,
            'P-MON-CKP', f.`P-MON-CKP`,
            'P-PDG', f.`P-PDG`
            )
        ) AS prediction
    FROM shm.3w.filter_well_data(p_well_number, p_num_minutes)
"""
)